In [56]:
import networkx as nx
import math
import igl
import matplotlib.pyplot as plt
import meshplot as mp

In [ ]:
import igl
import numpy as np
import networkx as nx

def compute_geodesic_distance(vertex1, vertex2):
    # Compute geodesic distance between two vertices
    # Replace this with your geodesic distance calculation using igl
    return np.linalg.norm(vertex1 - vertex2)

def build_weighted_graph_from_mesh(mesh_file):
    # Load mesh using igl
    mesh_vertices, mesh_faces = igl.read_triangle_mesh(mesh_file)

    # Initialize weighted graph
    weighted_graph = nx.Graph()

    # Add nodes for each vertex in the mesh
    for vertex in mesh_vertices:
        weighted_graph.add_node(tuple(vertex))

    # Cycle over the faces of the input mesh
    for face in mesh_faces:
        v0, v1, v2 = face  # Assuming each face is represented by three vertices

        # Add arcs joining each vertex of the face to the other two vertices of the face
        v0_coords, v1_coords, v2_coords = mesh_vertices[v0], mesh_vertices[v1], mesh_vertices[v2]
        weighted_graph.add_edge(tuple(v0_coords), tuple(v1_coords), weight=compute_geodesic_distance(v0_coords, v1_coords))
        weighted_graph.add_edge(tuple(v0_coords), tuple(v2_coords), weight=compute_geodesic_distance(v0_coords, v2_coords))
        weighted_graph.add_edge(tuple(v1_coords), tuple(v2_coords), weight=compute_geodesic_distance(v1_coords, v2_coords))

        # Find the neighbor face f' opposite to vertex v
        for neighbor_face in mesh_faces:
            if v0 in neighbor_face and v1 in neighbor_face and v2 in neighbor_face:
                continue  # Skip the current face as it's the same as the current face
            shared_edge = list(set(face) & set(neighbor_face))  # Find the shared edge between faces
            if len(shared_edge) == 2:
                v0_neighbor = list(set(neighbor_face) - set(shared_edge))[0]

                # Add arcs joining the current vertex to the vertex v0 belonging to the neighbor face
                weighted_graph.add_edge(tuple(mesh_vertices[v0]), tuple(mesh_vertices[v0_neighbor]), 
                                        weight=compute_geodesic_distance(mesh_vertices[v0], mesh_vertices[v0_neighbor]))

    return weighted_graph

# Example usage:
mesh_file_path = "data/bunny.off"  # Replace with the path to your mesh file
# Build the weighted graph from the mesh file
graph = build_weighted_graph_from_mesh(mesh_file_path)

# Print the nodes and edges with their weights
print("Nodes in the graph:", graph.nodes())
print("Edges in the graph with weights:")
for u, v, weight in graph.edges(data='weight'):
    print(f"Edge: {u} - {v}, Weight: {weight}")

In [ ]:
from collections import deque

# Function to compute geodesic distances using SLF-LLL algorithm
def shortest_paths(graph, sources):
    distances = {vertex: float('inf') for vertex in graph.nodes}
    queue = deque()

    # Add all sources to the queue with distance 0
    for source in sources:
        queue.appendleft(source)
        distances[source] = 0

    while queue:
        current_vertex = queue.pop()

        for neighbor in graph.neighbors(current_vertex):
            new_distance = distances[current_vertex] + graph[current_vertex][neighbor]['weight']
            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                if new_distance < distances[queue[-1]] if queue else False:
                    queue.append(neighbor)
                else:
                    queue.appendleft(neighbor)

    return distances

sources = [(-0.5, -0.5, 0.5), (0.5, -0.5, 0.5), (-0.5, 0.5, 0.5), (0.5, 0.5, 0.5), (-0.5, 0.5, -0.5), (0.5, 0.5, -0.5), (-0.5, -0.5, -0.5), (0.5, -0.5, -0.5)]

distances_dict = shortest_paths(graph,sources)

In [55]:
plt.figure(figsize=(8, 6))
viewer = mp.plot(cube_v, cube_f, c=distances_dict)

Invalid color array given! Supported are numpy arrays. <class 'dict'>


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

AttributeError: 'Viewer' object has no attribute 'show'

<Figure size 576x432 with 0 Axes>